In [148]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
seed = 42

In [149]:
train = pd.read_csv("train_agri.csv")
test_path = ""
test_data = pd.read_csv(test_path)
sub_path = "sample_submission.csv"
submission = pd.read_csv(sub_path)

In [150]:
train.shape

(88858, 10)

In [151]:
test_data.shape

(59310, 9)

In [152]:
train.head()

,ID,Estimated_Insects_Count,Crop_Type,Soil_Type,Pesticide_Use_Category,Number_Doses_Week,Number_Weeks_Used,Number_Weeks_Quit,Season,Crop_Damage
0,F00000001,188,1,0,1,0,0.0,0,1,0
1,F00000003,209,1,0,1,0,0.0,0,2,1
2,F00000004,257,1,0,1,0,0.0,0,2,1
3,F00000005,257,1,1,1,0,0.0,0,2,1
4,F00000006,342,1,0,1,0,0.0,0,2,1


In [153]:
train.corr(method ='pearson')

,Estimated_Insects_Count,Crop_Type,Soil_Type,Pesticide_Use_Category,Number_Doses_Week,Number_Weeks_Used,Number_Weeks_Quit,Season,Crop_Damage
Estimated_Insects_Count,1.000000,-0.088830,-0.056375,-0.079904,-0.102461,0.434925,0.299279,0.002304,0.202888
Crop_Type,-0.088830,1.000000,-0.287372,0.235779,-0.192161,-0.043991,-0.233394,0.002161,-0.016495
Soil_Type,-0.056375,-0.287372,1.000000,0.032657,0.079447,-0.070051,0.009364,0.002327,-0.021997
Pesticide_Use_Category,-0.079904,0.235779,0.032657,1.000000,-0.003498,0.344131,-0.554169,0.002811,0.175002
Number_Doses_Week,-0.102461,-0.192161,0.079447,-0.003498,1.000000,0.126426,-0.037131,-0.001401,-0.030874
Number_Weeks_Used,0.434925,-0.043991,-0.070051,0.344131,0.126426,1.000000,-0.449007,0.004417,0.232192
Number_Weeks_Quit,0.299279,-0.233394,0.009364,-0.554169,-0.037131,-0.449007,1.000000,0.000595,-0.133117
Season,0.002304,0.002161,0.002327,0.002811,-0.001401,0.004417,0.000595,1.000000,-0.000184
Crop_Damage,0.202888,-0.016495,-0.021997,0.175002,-0.030874,0.232192,-0.133117,-0.000184,1.000000


In [154]:
train['Crop_Damage'].value_counts()

0    74238
1    12307
2     2313
Name: Crop_Damage, dtype: int64

In [155]:
# Missing value in the dataset

train.isnull().sum()


ID                            0
Estimated_Insects_Count       0
Crop_Type                     0
Soil_Type                     0
Pesticide_Use_Category        0
Number_Doses_Week             0
Number_Weeks_Used          9000
Number_Weeks_Quit             0
Season                        0
Crop_Damage                   0
dtype: int64

In [156]:
test_data.isnull().sum()

ID                            0
Estimated_Insects_Count       0
Crop_Type                     0
Soil_Type                     0
Pesticide_Use_Category        0
Number_Doses_Week             0
Number_Weeks_Used          5893
Number_Weeks_Quit             0
Season                        0
dtype: int64

In [157]:
train['Number_Weeks_Used'] = train['Number_Weeks_Used'].fillna(train['Number_Weeks_Used'].mode()[0])

In [158]:
test_data['Number_Weeks_Used'] = test_data['Number_Weeks_Used'].fillna(test_data['Number_Weeks_Used'].mode()[0])

In [159]:
train.isnull().sum()

ID                         0
Estimated_Insects_Count    0
Crop_Type                  0
Soil_Type                  0
Pesticide_Use_Category     0
Number_Doses_Week          0
Number_Weeks_Used          0
Number_Weeks_Quit          0
Season                     0
Crop_Damage                0
dtype: int64

In [160]:
test_data.isnull().sum()

ID                         0
Estimated_Insects_Count    0
Crop_Type                  0
Soil_Type                  0
Pesticide_Use_Category     0
Number_Doses_Week          0
Number_Weeks_Used          0
Number_Weeks_Quit          0
Season                     0
dtype: int64

In [161]:
train.drop('ID',axis=1,inplace=True)

In [162]:
train.head()

,Estimated_Insects_Count,Crop_Type,Soil_Type,Pesticide_Use_Category,Number_Doses_Week,Number_Weeks_Used,Number_Weeks_Quit,Season,Crop_Damage
0,188,1,0,1,0,0.0,0,1,0
1,209,1,0,1,0,0.0,0,2,1
2,257,1,0,1,0,0.0,0,2,1
3,257,1,1,1,0,0.0,0,2,1
4,342,1,0,1,0,0.0,0,2,1


In [163]:
test_data.drop('ID',axis=1,inplace=True)

In [164]:
test_data.head()

,Estimated_Insects_Count,Crop_Type,Soil_Type,Pesticide_Use_Category,Number_Doses_Week,Number_Weeks_Used,Number_Weeks_Quit,Season
0,188,1,1,1,0,20.0,0,2
1,410,1,1,1,0,0.0,0,2
2,626,1,0,1,0,0.0,0,2
3,731,1,0,1,0,0.0,0,2
4,789,0,0,1,0,0.0,0,1


In [165]:
X_train, Y = train.drop(["Crop_Damage"], axis=1).values, train["Crop_Damage"].values
X_test = test_data.values

In [166]:
X_train.shape, Y.shape, X_test.shape

((88858, 8), (88858,), (59310, 8))

In [167]:
Y[:5]

array([0, 1, 1, 1, 1])

In [168]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_train, Y, test_size=0.2,random_state=seed)

# CATBOOST

In [140]:
!pip install catboost

     |████████████████████████████████| 66.3MB 61kB/s 


In [142]:
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from numpy import asarray
from numpy import mean
from numpy import std

CatB = CatBoostClassifier(verbose=0, n_estimators=100)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=seed)
n_scores = cross_val_score(CatB, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')

print('Accuracy: %.6f (%.6f)' % (mean(n_scores), std(n_scores)))

CatB.fit(X_train,y_train,early_stopping_rounds=5, 
             eval_set=[(X_test, y_test)], 
             verbose=False) 

CatB.score(X_train,y_train)

Accuracy: 0.846571 (0.001975)


0.8498719860450722

In [143]:
y_pred=CatB.predict(X_test)
CatB_acc = accuracy_score(y_test,y_pred)
CatB_mae = mean_absolute_error(y_pred,y_test) 
print("Mean Absolute Error:" , CatB_mae)
print("\nAccuracy Score {}".format(CatB_acc))

print("\nClassification report: \n{}".format(classification_report(y_test,y_pred)))

Mean Absolute Error: 0.1765136169255008

Accuracy Score 0.8427864055818141

Classification report: 
              precision    recall  f1-score   support

           0       0.86      0.99      0.92     14803
           1       0.53      0.15      0.24      2499
           2       0.00      0.00      0.00       470

    accuracy                           0.84     17772
   macro avg       0.46      0.38      0.38     17772
weighted avg       0.79      0.84      0.80     17772



In [177]:
result = CatB.predict(test_data)

submission = pd.read_csv(sub_path)
submission = submission.drop('Crop_Damage',axis=1)
submission['Crop_Damage'] = result

In [179]:
submission.to_csv('CatBoost_submission.csv')

In [181]:
submission.head()

,ID,Crop_Damage
0,F00000002,1
1,F00000011,1
2,F00000014,1
3,F00000015,1
4,F00000016,1


# LGBM

In [182]:
!pip install lightgbm

In [183]:
from lightgbm import LGBMClassifier

model_lgbm = LGBMClassifier(verbose=0, n_estimators=1000,learning_rate=0.05, n_jobs=4)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=seed)
n_scores = cross_val_score(model_lgbm, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')

print('Accuracy: %.6f (%.6f)' % (mean(n_scores), std(n_scores)))

model_lgbm.fit(X_train,y_train,early_stopping_rounds=5, 
             eval_set=[(X_test, y_test)], 
             verbose=False) 

model_lgbm.score(X_train,y_train)

Accuracy: 0.846275 (0.002619)


0.8508989111780098

In [185]:
y_pred=model_lgbm.predict(X_test)
lgbm_acc = accuracy_score(y_test,y_pred)
lgbm_mae = mean_absolute_error(y_pred,y_test) 
print("Mean Absolute Error:" , lgbm_mae)
print("\nAccuracy Score {}".format(lgbm_acc))

print("\nClassification report: \n{}".format(classification_report(y_test,y_pred)))

Mean Absolute Error: 0.17482556830970064

Accuracy Score 0.8441368444744543

Classification report: 
              precision    recall  f1-score   support

           0       0.86      0.98      0.92     14803
           1       0.54      0.17      0.26      2499
           2       0.00      0.00      0.00       470

    accuracy                           0.84     17772
   macro avg       0.47      0.38      0.39     17772
weighted avg       0.79      0.84      0.80     17772



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
